In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
pip install scikit-learn

In [ ]:
pip install sumy networkx

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 59.0 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=dc6d54f069108e05d05808626920084eebfb475a2ddd231b33dd2447c404b78c
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=107876ca3de71f2e11881b7a5b629dd6ed4f9eacbbfbbfd80a7cc2f96a13e831
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [ ]:
pip install torch

In [ ]:
pip install transformers


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

importing important libraries and dependencies


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import networkx as nx
import numpy as np

frequency based method

In [ ]:
# Frequency-based summarization function
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
def frequency_based_summarization(text, num_sentences):
    words = word_tokenize(text.lower())
    sentences = sent_tokenize(text)
    stop_words = set(stopwords.words("english"))

    freq_table = {}
    for word in words:
        if word not in stop_words and word not in string.punctuation:
            freq_table[word] = freq_table.get(word, 0) + 1

    sentence_scores = {}
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in freq_table:
                sentence_scores[sentence] = sentence_scores.get(sentence, 0) + freq_table[word]

    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    summary = " ".join(summary_sentences)
    return summary


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


LSA (LATENT SEMANTIC ANALYSIS)

In [ ]:
# LSA summarization function
def lsa_summarization(text, num_sentences):
    sentences = sent_tokenize(text)
    tfidf_vectorizer = TfidfVectorizer(stop_words="english")
    sentence_matrix = tfidf_vectorizer.fit_transform(sentences)

    # Applying SVD to reduce dimensionality and get the main topics
    svd = TruncatedSVD(n_components=1)
    svd.fit(sentence_matrix)
    scores = svd.transform(sentence_matrix).flatten()

    # Select top sentences based on scores
    ranked_sentences = [sentences[i] for i in np.argsort(scores)[::-1]]
    summary = " ".join(ranked_sentences[:num_sentences])
    return summary

Textrank method

In [ ]:
# TextRank summarization function
def textrank_summarization(text, num_sentences):
    sentences = sent_tokenize(text)
    tfidf_vectorizer = TfidfVectorizer(stop_words="english")
    sentence_matrix = tfidf_vectorizer.fit_transform(sentences)

    similarity_matrix = (sentence_matrix * sentence_matrix.T).toarray()
    graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(graph)

    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    summary = " ".join([s for _, s in ranked_sentences[:num_sentences]])
    return summary

# Abstractive summarization

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, BartForConditionalGeneration, BartTokenizer, PegasusForConditionalGeneration, PegasusTokenizer
import torch
import gradio as gr

# Load the T5 model and tokenizer
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small", legacy=False)

# Load the BART model and tokenizer
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Define Abstractive Summarization functions with word limit
def t5_summarization(text, max_words):
    inputs = t5_tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = t5_model.generate(inputs.input_ids, max_length=max_words, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def bart_summarization(text, max_words):
    inputs = bart_tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = bart_model.generate(inputs.input_ids, max_length=max_words, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## LLM SUMMARIZATION

In [ ]:
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
%pip install --upgrade --quiet tiktoken langchain langgraph beautifulsoup4 langchain langchain-google-genai langchain-huggingface

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

def load_llm(model="gemini-1.5-flash"):
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2)
    return llm

In [ ]:
from langchain_core.prompts import ChatPromptTemplate


def get_prompt_template():
  # Define prompt
  prompt = ChatPromptTemplate.from_messages(
      [
          (
              "system",
              "Write a concise summary of the following in {num_words} words:\\n\\n",
          ),
          ("human", "{context}")
      ]
  )

  return prompt


def LLM_summarization(text, num_words=50, model="gemini-1.5-flash"):

  # Load LLM
  llm = load_llm(model)

  # Get Prompt Template
  prompt = get_prompt_template()

  # Instantiate chain
  chain = prompt | llm

  # Invoke chain
  result = chain.invoke({
        "context": text,
        "num_words": num_words
      })

  # Return result
  return result.content

In [ ]:
# Specifcy the text and number of words
summary = summarize_text(text, num_words = 50, model = "gemini-1.5-flash")

NameError: name 'summarize_text' is not defined

In [ ]:
# Summarization pipeline function
def summarization_pipeline(text, summ_type, model_choice, num_sentences=None, max_words=None):
    original_length = len(text.split())
    if summ_type == "Extractive":
        if model_choice == "Frequency-based":
            return frequency_based_summarization(text, num_sentences)
        elif model_choice == "LSA":
            return lsa_summarization(text, num_sentences)
        elif model_choice == "TextRank":
            return textrank_summarization(text, num_sentences)
    elif summ_type == "Abstractive":
        if model_choice == "T5":
            summary = t5_summarization(text, max_words)
        elif model_choice == "BART":
            summary = bart_summarization(text, max_words)
        elif model_choice == "LLM":
            summary = LLM_summarization(text, max_words)
        else:
            summary = "Model not available"
        summary_length = len(summary.split())
        return f"Original Length: {original_length} words\nSummary Length: {summary_length} words\n\n{summary}"

# Define Gradio Interface
with gr.Blocks() as interface:
    gr.Markdown("# Summarization Interface")

    # Select summarization type
    summ_type = gr.Radio(["Extractive", "Abstractive"], label="Choose summarization type")

    # Options for Extractive and Abstractive methods
    model_choice_extractive = gr.Dropdown(["Frequency-based", "LSA", "TextRank"], visible=False, label="Choose extractive summarization method")
    model_choice_abstractive = gr.Dropdown(["T5", "BART", "LLM"], visible=False, label="Choose abstractive summarization model")

    # Textbox for text input
    text_input = gr.Textbox(lines=5, placeholder="Enter text to summarize", label="Input Text")

    # Input for number of lines or words based on summarization type
    num_sentences_input = gr.Slider(1, 20, step=1, label="Number of sentences for extractive summary", visible=False)
    num_words_input = gr.Slider(10, 500, step=10, label="Word limit for abstractive summary", visible=False)

    # Dynamically update the visibility of model choice and length inputs
    def update_inputs(summ_type):
        if summ_type == "Extractive":
            return gr.update(visible=True), gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)
        else:
            return gr.update(visible=False), gr.update(visible=True), gr.update(visible=False), gr.update(visible=True)

    summ_type.change(update_inputs, [summ_type], [model_choice_extractive, model_choice_abstractive, num_sentences_input, num_words_input])

    # Generate button and output text box
    generate_button = gr.Button("Generate Summary")
    output_text = gr.Textbox(label="Generated Summary")

    # Connects the input to the summarization pipeline and updates the output
    def generate_summary(text, summ_type, model_choice_extractive, model_choice_abstractive, num_sentences, num_words):
        model_choice = model_choice_extractive if summ_type == "Extractive" else model_choice_abstractive
        return summarization_pipeline(text, summ_type, model_choice, num_sentences=num_sentences, max_words=num_words)

    generate_button.click(generate_summary, [text_input, summ_type, model_choice_extractive, model_choice_abstractive, num_sentences_input, num_words_input], output_text)

# Launch the Gradio interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0ee4c4c95a49a984af.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
